# Navigation

---

### --- Start the Project --- ###


In [1]:
from unityagents import UnityEnvironment
from collections import deque
from dqn_agent import Agent

import numpy as np
import os
import torch
import matplotlib.pyplot as plt
%matplotlib inline

Scenario where the project will be executed,configure according to the operating system

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
## Training Agent
def dqn(n_episodes=1930, max_t=3000, eps_start=1.0, eps_end=0.01, eps_decay=0.997):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []       # list containing scores from each episode
    scores_mean = []  # list the mean of the window scores
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset(train_mode=True)[brain_name].vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        
            next_state = env_info.vector_observations[0]   
            reward = env_info.rewards[0]                   
            done = env_info.local_done[0]                  
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        
        scores.append(score)              # save most recent score
        scores_window.append(score)       # save most recent score
        scores_mean.append(np.mean(scores_window))
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, scores_mean[-1]), end="")
        
        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}' .format(i_episode, scores_mean[-1]))
        if np.mean(scores_window)>=13.0:
            print('\nSolved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores, scores_mean

In [3]:
### Create and train the Agent

## Create the environment
env = UnityEnvironment('Banana_Linux/Banana.x86_64')

# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Get the environment info
env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
state = env_info.vector_observations[0]
state_size = len(state)

## Creating Agent
agent = Agent(state_size=state_size, action_size=action_size, seed=0, lr_decay=0.9999)

## Training Agent
scores, mean = dqn(n_episodes=400, max_t=300, eps_start=0.10, eps_end=0.01, eps_decay=0.987)

## Monitoring Scores
fig = plt.figure(figsize=(20,10))

ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.plot(np.arange(len(mean)), mean)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(('Score', 'Mean'), fontsize='xx-large')

plt.show()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Running on: cpu


NameError: name 'DuelingQNetwork' is not defined

In [ ]:
## Create the environment
try:
    env
except:
    env = UnityEnvironment('Banana_Linux/Banana.x86_64')

    # Get the default brain
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]

    # Get the environment info
    env_info = env.reset(train_mode=False)[brain_name]
    action_size = brain.vector_action_space_size
    state = env_info.vector_observations[0]
    state_size = len(state)

    ## Creating Agent
    agent = Agent(state_size=state_size, action_size=action_size, seed=0)

## Load model
state_dict = torch.load('checkpoint.pth')
agent.qnetwork_local.load_state_dict(state_dict)
agent.qnetwork_local.eval()


n_episodes = 5
max_t = 300

## Trainig Agent
scores = []      
scores_mean = []  
for i_episode in range(1, n_episodes+1):
    state = env.reset(train_mode=False)[brain_name].vector_observations[0]
    score = 0
    for t in range(max_t):
        action = agent.act(state)
        env_info = env.step(action)[brain_name]        
        next_state = env_info.vector_observations[0]  
        reward = env_info.rewards[0]                  
        done = env_info.local_done[0]                
        state = next_state
        score += reward
        print('\rEpisode {}\Score: {:.2f}'.format(i_episode, score), end="")
        if done:
            break 

    scores.append(score)             
    scores_mean.append(np.mean(scores))

    print('\rEpisode {}\Score: {:.2f}'.format(i_episode, scores[-1]))

In [ ]:
## Close the environment
env.close()